# Sección nueva

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# ======================================================================
# 1. Configuración
# ======================================================================
COLUMNAS_OBJETIVO = [
    'hdl_cholesterol_human',
    'hemoglobin(hgb)_human'
]

ARCHIVOS = {
    'train': 'Train.csv',
    'test': 'Test.csv'
}

# ======================================================================
# 2. Funciones de Limpieza de Datos Mejoradas
# ======================================================================
def extraer_numeros(valor):
    """Extrae valores numéricos de cadenas complejas"""
    try:
        coincidencias = re.findall(r'-?\d+\.?\d*', str(valor))
        return float(coincidencias[0]) if coincidencias else np.nan
    except:
        return np.nan

def limpiar_columna(df, col):
    """Limpieza avanzada para columnas objetivo"""
    print(f"Valores originales en {col}: {df[col].unique()[:10]}")
    df[col] = pd.to_numeric(df[col], errors='coerce')
    print(f"Valores después de conversión a numérico en {col}: {df[col].unique()[:10]}")
    df[col] = df[col].fillna(df[col].median())
    return df

def procesar_numericos(df):
    """Procesamiento completo de columnas numéricas"""
    print("\n=== LIMPIEZA DE DATOS ===")

    # Procesar columnas objetivo
    for col in COLUMNAS_OBJETIVO:
        if col in df.columns:
            print(f"\nProcesando {col}:")
            df[col] = df[col].astype(str)  # Asegurarse de que sea una cadena
            df[col] = df[col].replace({'ok': 1, 'low': 0, 'high': 2})  # Codificación manual
            df[col] = pd.to_numeric(df[col], errors='coerce')  # Convertir a numérico
            df[col] = df[col].fillna(df[col].median())  # Imputar NaNs
            print(f"NaNs: {df[col].isna().sum()}")

    # Procesar absorbancias y sensores
    columnas_numericas = [f'absorbance{i}' for i in range(170)] + ['temperature', 'humidity']
    for col in columnas_numericas:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
            print(f"NaNs en {col} antes de imputación: {df[col].isna().sum()}")
            df[col] = df[col].fillna(df[col].median())
            print(f"NaNs en {col} después de imputación: {df[col].isna().sum()}")

    return df

# ======================================================================
# 3. Pipeline de Procesamiento
# ======================================================================
def cargar_y_procesar(ruta, pca=None, scaler=None, es_entrenamiento=False):
    try:
        df = pd.read_csv(ruta)
        print(f"\n=== PROCESANDO {ruta} ===")
        print(f"Registros : {len(df):,}")

        # Limpieza avanzada
        df = procesar_numericos(df)

        # Reducción dimensional
        absorb_cols = [f'absorbance{i}' for i in range(170)]
        if es_entrenamiento or pca is None:
            pca = PCA(n_components=0.95)
            componentes = pca.fit_transform(df[absorb_cols])
        else:
            componentes = pca.transform(df[absorb_cols])

        df = pd.concat([df.drop(columns=absorb_cols),
                       pd.DataFrame(componentes, columns=[f'pca_{i}' for i in range(componentes.shape[1])])],
                      axis=1)

        # Escalado
        features = [col for col in df.columns if col.startswith('pca_')] + ['temperature', 'humidity']
        if scaler:
            X = scaler.transform(df[features]) if not es_entrenamiento else scaler.fit_transform(df[features])
        else:
            scaler = RobustScaler()
            X = scaler.fit_transform(df[features])

        # Extraer objetivos
        y = df[COLUMNAS_OBJETIVO] if es_entrenamiento else None

        return X, y, pca, scaler, df['Reading_ID']

    except Exception as e:
        print(f"Error procesando {ruta}: {str(e)}")
        raise

# ======================================================================
# 4. Modelo de Deep Learning Optimizado
# ======================================================================
def construir_modelo(input_dim):
    inputs = Input(shape=(input_dim,))

    # Capas ocultas
    x = Dense(128, activation='relu', kernel_initializer='he_normal')(inputs)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)

    x = Dense(64, activation='relu', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)

    # Salidas
    salida_0 = Dense(1, name='salida_0')(x)
    salida_1 = Dense(1, name='salida_1')(x)

    model = Model(inputs=inputs, outputs=[salida_0, salida_1])
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='mean_squared_error',
                  metrics=['mae', 'mae'])  # Especificar métricas para ambas salidas
    return model

# ======================================================================
# 5. Entrenamiento del Modelo
# ======================================================================
def entrenar_modelo(X_train, y_train):
    model = construir_modelo(X_train.shape[1])
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    history = model.fit(X_train,
                        [y_train[col] for col in COLUMNAS_OBJETIVO],
                        validation_split=0.2,
                        epochs=100,
                        batch_size=32,
                        callbacks=[early_stopping],
                        verbose=1)
    return model, history

# ======================================================================
# 6. Predicción y Guardado de Resultados
# ======================================================================
def predecir_y_guardar(model, X_test, ids):
    predicciones = model.predict(X_test)

    # Convertir las predicciones a un DataFrame
    resultados = pd.DataFrame({
        COLUMNAS_OBJETIVO[0]: predicciones[0].flatten(),  # Salida 1
        COLUMNAS_OBJETIVO[1]: predicciones[1].flatten()   # Salida 2
    })

    resultados['Reading_ID'] = ids
    resultados.to_csv('predicciones.csv', index=False)
    print("Predicciones guardadas en 'predicciones.csv'.")

# ======================================================================
# 7. Ejecución del Pipeline Completo
# ======================================================================
if __name__ == "__main__":
    # Cargar y procesar datos
    X_train, y_train, pca, scaler, _ = cargar_y_procesar(ARCHIVOS['train'], es_entrenamiento=True)
    X_test, _, _, _, ids_test = cargar_y_procesar(ARCHIVOS['test'], pca=pca, scaler=scaler)

    # Entrenar modelo
    model, history = entrenar_modelo(X_train, y_train)

    # Predecir y guardar resultados
    predecir_y_guardar(model, X_test, ids_test)


=== PROCESANDO Train.csv ===
Registros : 13,140

=== LIMPIEZA DE DATOS ===

Procesando hdl_cholesterol_human:
NaNs: 0

Procesando hemoglobin(hgb)_human:
NaNs: 0
NaNs en absorbance0 antes de imputación: 0
NaNs en absorbance0 después de imputación: 0
NaNs en absorbance1 antes de imputación: 0
NaNs en absorbance1 después de imputación: 0
NaNs en absorbance2 antes de imputación: 0
NaNs en absorbance2 después de imputación: 0
NaNs en absorbance3 antes de imputación: 0
NaNs en absorbance3 después de imputación: 0
NaNs en absorbance4 antes de imputación: 0
NaNs en absorbance4 después de imputación: 0
NaNs en absorbance5 antes de imputación: 0
NaNs en absorbance5 después de imputación: 0
NaNs en absorbance6 antes de imputación: 0
NaNs en absorbance6 después de imputación: 0
NaNs en absorbance7 antes de imputación: 0
NaNs en absorbance7 después de imputación: 0
NaNs en absorbance8 antes de imputación: 0
NaNs en absorbance8 después de imputación: 0
NaNs en absorbance9 antes de imputación: 0
NaN

<ipython-input-9-85c93566b03f>:53: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].replace({'ok': 1, 'low': 0, 'high': 2})  # Codificación manual
<ipython-input-9-85c93566b03f>:53: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].replace({'ok': 1, 'low': 0, 'high': 2})  # Codificación manual


NaNs en absorbance47 después de imputación: 0
NaNs en absorbance48 antes de imputación: 0
NaNs en absorbance48 después de imputación: 0
NaNs en absorbance49 antes de imputación: 0
NaNs en absorbance49 después de imputación: 0
NaNs en absorbance50 antes de imputación: 0
NaNs en absorbance50 después de imputación: 0
NaNs en absorbance51 antes de imputación: 0
NaNs en absorbance51 después de imputación: 0
NaNs en absorbance52 antes de imputación: 0
NaNs en absorbance52 después de imputación: 0
NaNs en absorbance53 antes de imputación: 0
NaNs en absorbance53 después de imputación: 0
NaNs en absorbance54 antes de imputación: 0
NaNs en absorbance54 después de imputación: 0
NaNs en absorbance55 antes de imputación: 0
NaNs en absorbance55 después de imputación: 0
NaNs en absorbance56 antes de imputación: 0
NaNs en absorbance56 después de imputación: 0
NaNs en absorbance57 antes de imputación: 0
NaNs en absorbance57 después de imputación: 0
NaNs en absorbance58 antes de imputación: 0
NaNs en ab